# Imbalance mapping analysis

Let's get down to it! This module is relatively easy to use, but the explanation are slightly more complex.

## Practice data

As always, `sihnpy` comes with data from the PREVENT-AD to be used as practice. In this case, **structural MRI data, preprocessed with FreeSurfer** is provided (see the {ref}`PREVENT-AD preprocessing documentation for more info <0.pad_data/datasets_usage:Additional information on brain imaging preprocessing>`).

## Imbalance mapping

### 1. Preparing the data

**Imbalance mapping** only requires:
* A `pandas.DataFrame` where 
    * The index is the participants' IDs
    * The columns contain the variables you want to run the **imbalance mapping** on (e.g., thickness/volume).

If you already have your data, you can skip ahead to the {ref}`next section <4.imbalance_mapping/imb_module:2. Imbalance mapping>`.

Loading the data is super easy:


In [1]:
from sihnpy.datasets import pad_imb_input

volume_data, thickness_data, aseg_data = pad_imb_input()
thickness_data

,session,run,ctx_lh_bankssts_thickness,ctx_lh_caudalanteriorcingulate_thickness,ctx_lh_caudalmiddlefrontal_thickness,ctx_lh_cuneus_thickness,ctx_lh_entorhinal_thickness,ctx_lh_fusiform_thickness,ctx_lh_inferiorparietal_thickness,ctx_lh_inferiortemporal_thickness,...,ctx_rh_rostralanteriorcingulate_thickness,ctx_rh_rostralmiddlefrontal_thickness,ctx_rh_superiorfrontal_thickness,ctx_rh_superiorparietal_thickness,ctx_rh_superiortemporal_thickness,ctx_rh_supramarginal_thickness,ctx_rh_frontalpole_thickness,ctx_rh_temporalpole_thickness,ctx_rh_transversetemporal_thickness,ctx_rh_insula_thickness
participant_id,,,,,,,,,,,,,,,,,,,,,
sub-1000173,ses-FU12,run-001,2.153,2.334,2.451,1.767,3.202,2.452,2.161,2.515,...,2.387,2.221,2.424,2.092,2.487,2.357,2.403,3.432,2.175,2.758
sub-1002928,ses-BL00,run-001,2.462,2.403,2.449,1.735,3.316,2.664,2.253,2.586,...,2.577,2.043,2.492,2.025,2.609,2.402,2.521,3.469,2.418,2.887
sub-1004359,ses-BL00,run-001,2.246,2.276,2.346,1.815,3.332,2.728,2.388,2.620,...,2.776,2.327,2.515,2.157,2.733,2.437,3.009,3.710,2.276,2.918
sub-1004359,ses-FU12,run-001,2.238,2.339,2.349,1.827,3.449,2.674,2.403,2.610,...,2.827,2.430,2.551,2.112,2.719,2.438,2.778,3.698,2.342,2.938
sub-1016072,ses-BL00,run-001,2.213,2.544,2.327,1.662,3.310,2.580,2.295,2.603,...,2.440,2.224,2.435,2.084,2.583,2.461,2.710,3.735,2.302,2.810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sub-9930257,ses-BL00,run-001,2.435,2.476,2.511,1.893,2.903,2.557,2.397,2.645,...,2.825,2.249,2.656,2.183,2.729,2.474,2.620,3.617,2.629,2.890
sub-9931234,ses-BL00,run-001,2.461,2.166,2.526,1.755,3.143,2.618,2.383,2.918,...,2.606,2.252,2.500,2.200,2.710,2.466,2.496,3.257,2.497,2.879
sub-9931234,ses-FU12,run-001,2.403,2.194,2.536,1.756,3.216,2.594,2.345,2.837,...,2.660,2.256,2.518,2.213,2.750,2.435,2.632,3.380,2.566,2.742


As I mentioned {ref}`in the introduction <4.imbalance_mapping/imb_intro:Data type>`, `sihnpy` assumes that the data has a single row for each participant. However, to give you more practice opportunities, `sihnpy` data for this module comes with two visits: BL00 and FU12. For now, we will restrict the data to only the baseline visits and drop the "session" and "run" variables as they aren't needed.

In [2]:
thickness_data_final = thickness_data[thickness_data['session'] == 'ses-BL00'].drop(labels=['session', 'run'], axis=1)
thickness_data_final #We get the 68 columns of data needed

,ctx_lh_bankssts_thickness,ctx_lh_caudalanteriorcingulate_thickness,ctx_lh_caudalmiddlefrontal_thickness,ctx_lh_cuneus_thickness,ctx_lh_entorhinal_thickness,ctx_lh_fusiform_thickness,ctx_lh_inferiorparietal_thickness,ctx_lh_inferiortemporal_thickness,ctx_lh_isthmuscingulate_thickness,ctx_lh_lateraloccipital_thickness,...,ctx_rh_rostralanteriorcingulate_thickness,ctx_rh_rostralmiddlefrontal_thickness,ctx_rh_superiorfrontal_thickness,ctx_rh_superiorparietal_thickness,ctx_rh_superiortemporal_thickness,ctx_rh_supramarginal_thickness,ctx_rh_frontalpole_thickness,ctx_rh_temporalpole_thickness,ctx_rh_transversetemporal_thickness,ctx_rh_insula_thickness
participant_id,,,,,,,,,,,,,,,,,,,,,
sub-1002928,2.462,2.403,2.449,1.735,3.316,2.664,2.253,2.586,2.130,2.116,...,2.577,2.043,2.492,2.025,2.609,2.402,2.521,3.469,2.418,2.887
sub-1004359,2.246,2.276,2.346,1.815,3.332,2.728,2.388,2.620,2.258,2.150,...,2.776,2.327,2.515,2.157,2.733,2.437,3.009,3.710,2.276,2.918
sub-1016072,2.213,2.544,2.327,1.662,3.310,2.580,2.295,2.603,1.986,2.035,...,2.440,2.224,2.435,2.084,2.583,2.461,2.710,3.735,2.302,2.810
sub-1031654,2.455,2.340,2.355,1.842,3.381,2.833,2.290,2.653,2.324,2.194,...,2.643,2.307,2.628,2.052,2.692,2.441,3.035,3.873,2.224,3.056
sub-1072774,2.403,2.480,2.485,1.752,3.443,2.672,2.356,2.890,2.561,2.057,...,2.760,2.247,2.593,2.147,2.577,2.451,2.315,3.758,2.195,3.140
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sub-9889544,2.438,2.438,2.441,1.838,3.306,2.613,2.315,2.682,2.169,2.218,...,2.644,2.227,2.532,2.271,2.863,2.475,2.226,3.697,2.561,2.727
sub-9909448,2.502,2.062,2.359,1.750,3.127,2.642,2.314,2.675,1.962,2.073,...,2.524,2.311,2.508,2.043,2.664,2.384,2.477,3.666,2.634,2.769
sub-9930257,2.435,2.476,2.511,1.893,2.903,2.557,2.397,2.645,2.280,2.080,...,2.825,2.249,2.656,2.183,2.729,2.474,2.620,3.617,2.629,2.890


### 2. Imbalance mapping

Ok now is the fun part: the **imbalance mapping**. The code behind is complex, but the application is simple. The function only requires two input:
* A `pandas.DataFrame` with the index as the participant IDs and the columns being the variables we want to map.
* The `type` of orthogonal distances we want to get.

The `type` of distances is simply whether we want **absolute** (`abs`) distances or **signed** (`sign`) distances from the regression slope. In the **absolute** version, we don't care whether the fitted value is above or below the diagonal: we just want to know how far the value is. In the **signed** version, we want to know whether the fitted value was overestimated (i.e., above the regression line) or underestimated (i.e., below the regression line). Whether you want **absolute** or **signed** distances is really up to you. In the original article by Nadig et al. (2021)[^Nadig_2021], the **absolute** residuals were used. Here is the code to run it:

In [3]:
from sihnpy import imbalance_mapping as imb

residual_array = imb.imbalance_mapping(data=thickness_data_final, type='abs')

Computing region 1/68
Computing region 2/68
Computing region 3/68
Computing region 4/68
Computing region 5/68
Computing region 6/68
Computing region 7/68
Computing region 8/68
Computing region 9/68
Computing region 10/68
Computing region 11/68
Computing region 12/68
Computing region 13/68
Computing region 14/68
Computing region 15/68
Computing region 16/68
Computing region 17/68
Computing region 18/68
Computing region 19/68
Computing region 20/68
Computing region 21/68
Computing region 22/68
Computing region 23/68
Computing region 24/68
Computing region 25/68
Computing region 26/68
Computing region 27/68
Computing region 28/68
Computing region 29/68
Computing region 30/68
Computing region 31/68
Computing region 32/68
Computing region 33/68
Computing region 34/68
Computing region 35/68
Computing region 36/68
Computing region 37/68
Computing region 38/68
Computing region 39/68
Computing region 40/68
Computing region 41/68
Computing region 42/68
Computing region 43/68
Computing region 44/

So, what did we do, and what did we get? 

In this step, we do the equivalent of step B) and C) from {ref}`Nadig et al. (2021)'s figure <4.imbalance_mapping/imb_intro:Definitions>`: we compute the structural covariance and extract the **orthogonal distances** for each participant. This is store in a massive 3D matrix (well... I guess it's a rectangle at this point):

In [4]:
residual_array.shape

(68, 68, 306)

Here's a simplified illustration of what it would look like if we illustrated this:

```{figure} ../images/imb/resid_array_rep.png
:name: Residual array - Illustration
:scale: 30
:align: center
```
<br>

In this cube, each square matrix represent the orthogonal distances from the regression of the values of two regions for a single participant. The diagonal is removed (in the image it is whited out) as the correlation of the same column twice would give a perfect correlation.

While the shape of the image is much smaller than the actual matrix (68 regions X 68 regions X 306 participants), the principle is exactly the same.

Once this 3D matrix is calculated, we are already ready to compute our measures.

### 3. Imbalance statistics

In the original code by Nadig et al. (2021)[^Nadig_2021] proposes two measures: computing an average imbalance by person (i.e., average imbalance across all edges) and an average imbalance by region at the group-level (i.e., how much imbalance does the entorhinal cortex show for example). `sihnpy` integrates both measures, but also outputs an average imbalance by person by region (i.e., how much imbalance on average the entorhinal cortex shows in person Y). Below is a quick illustration of these measures:

```{figure} ../images/imb/imb_measures.png
:name: Imbalance mapping measures
:scale: 30
:align: center
```

These measures in `sihnpy` can be calculated very easily with a single function taking two arguments:
* The original `pandas.DataFrame` used to compute the **imbalance mapping** (we use it to grab the participant IDs and the region names)
* The cube of orthogonal distances we computed in the previous step

In [5]:
avg_imb_by_region, avg_imb_by_person, avg_imb_by_pers_by_region = imb.imbalance_stats(data=thickness_data_final, residual_array=residual_array)
avg_imb_by_pers_by_region

,ctx_lh_bankssts_thickness,ctx_lh_caudalanteriorcingulate_thickness,ctx_lh_caudalmiddlefrontal_thickness,ctx_lh_cuneus_thickness,ctx_lh_entorhinal_thickness,ctx_lh_fusiform_thickness,ctx_lh_inferiorparietal_thickness,ctx_lh_inferiortemporal_thickness,ctx_lh_isthmuscingulate_thickness,ctx_lh_lateraloccipital_thickness,...,ctx_rh_rostralanteriorcingulate_thickness,ctx_rh_rostralmiddlefrontal_thickness,ctx_rh_superiorfrontal_thickness,ctx_rh_superiorparietal_thickness,ctx_rh_superiortemporal_thickness,ctx_rh_supramarginal_thickness,ctx_rh_frontalpole_thickness,ctx_rh_temporalpole_thickness,ctx_rh_transversetemporal_thickness,ctx_rh_insula_thickness
participant_id,,,,,,,,,,,,,,,,,,,,,
sub-1002928,0.050868,0.070757,0.045348,0.131103,0.090582,0.031317,0.051684,0.129795,0.064045,0.060393,...,0.068125,0.135052,0.065608,0.109356,0.045579,0.079806,0.069990,0.066805,0.066181,0.055967
sub-1004359,0.108628,0.077152,0.080876,0.046602,0.079860,0.047387,0.043844,0.062625,0.063184,0.030747,...,0.084405,0.057178,0.032863,0.028792,0.048641,0.036213,0.106226,0.087844,0.076241,0.052139
sub-1016072,0.153541,0.107828,0.115252,0.146310,0.103709,0.076043,0.052581,0.096220,0.122770,0.121347,...,0.109038,0.055124,0.100050,0.076545,0.084167,0.045742,0.106502,0.111446,0.104460,0.073354
sub-1031654,0.059193,0.076811,0.079381,0.051086,0.083903,0.122421,0.063430,0.054097,0.080342,0.043770,...,0.074475,0.055412,0.086878,0.091662,0.046471,0.039275,0.108813,0.095604,0.096446,0.109220
sub-1072774,0.042627,0.075648,0.045346,0.077476,0.082345,0.032254,0.027092,0.146982,0.140083,0.094995,...,0.080289,0.039019,0.051833,0.029578,0.097529,0.030800,0.105583,0.080944,0.091024,0.153880
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sub-9889544,0.047168,0.063802,0.037739,0.045298,0.073720,0.063764,0.037288,0.041076,0.057354,0.052754,...,0.067646,0.048609,0.030474,0.097550,0.132909,0.032159,0.100336,0.081138,0.083595,0.106776
sub-9909448,0.065361,0.116467,0.081611,0.086068,0.089651,0.043559,0.036690,0.044671,0.114514,0.081760,...,0.082605,0.052499,0.040859,0.095083,0.046149,0.076434,0.090830,0.089720,0.106595,0.079453
sub-9930257,0.056596,0.089051,0.057623,0.055001,0.106769,0.101592,0.058074,0.063256,0.078726,0.079335,...,0.112533,0.043268,0.098764,0.040644,0.056836,0.039443,0.092014,0.096737,0.110040,0.056424


And that's it! You have everything you need to run the analysis on your own.

```{admonition} Advanced topic: Subsetting specific regions for averaging
:class: danger

In some cases, you may want to check the average imbalance in specific, pre-defined networks for instance. For instance, let's say you are investigating the **imbalance mapping** specifically in the temporal lobe. In such a case, you may be interested in computing the imbalance only in a specific set of regions. However, `sihnpy` doesn't yet natively support this. Two options are offered to you in this case:

1) Either you restrict your data (i.e., `thickness_data_final`) to include only the columns you are interested in.
2) You manipulate the `residual_array` to select only the parts of the rectangle that interest you.

The first solution is probably easiest, but it will require you to recompute the **imbalance mapping** for every different set of regions you want which may get computationally expensive when running it on many regions and/or combinations of regions.

The second solution is more complicated to do and require good knowledge of a [array slicing in Numpy](https://numpy.org/doc/stable/user/basics.indexing.html). It can take some getting used to, particularly when working in 3D. However, in this method, you only need to compute **imbalance mapping** once, and then you just need to feed the different slices to `imbalance_stats` which is much less computationally expensive.

If there is enough interest, I will adapt a way to do this natively in `sihnpy`. [Open an issue](https://github.com/stong3/sihnpy/issues) if you are interested!
```

### 4. Export

The next function needs a lot of arguments, but the goal is very simple: output the different measures computed during the **imbalance mapping** analysis. To output, simply follow the following command:

```python
imb.export(data=thickness_data_final, residual_array=residual_array, output_path="/path/to/output", avg_imb_by_region=avg_imb_by_region, avg_imb_by_person=avg_imb_by_person, avg_imb_by_pers_by_region=avg_imb_by_pers_by_region, name='test', all=False)
```

The only choice the user has over is whether we should export the matrices of orthogonal distances for each participant. This could be useful for other analyses but will require a lot more space. By default, this is not output, but the user can choose to do so by switching `False` to `True`.

## tl;dr

Too lazy to read everything? Or read everything and need a quick refresher? Here is the code in the order you need to make it work.

```python
from sihnpy.datasets import pad_imb_input #For practice data
from sihnpy import imbalance_mapping as imb #Imbalance mapping functions

volume_data, thickness_data, aseg_data = pad_imb_input() #Import the practice data
#If the data has more than 1 row by participant, clean the data
thickness_data_final = thickness_data[thickness_data['session'] == 'ses-BL00'].drop(labels=['session', 'run'], axis=1)

#Imbalance mapping
residual_array = imb.imbalance_mapping(data=thickness_data_final, type='abs')
avg_imb_by_region, avg_imb_by_person, avg_imb_by_pers_by_region = imb.imbalance_stats(data=thickness_data_final, residual_array=residual_array)

#Export
imb.export(data=thickness_data_final, residual_array=residual_array, output_path="/path/to/output", avg_imb_by_region=avg_imb_by_region, avg_imb_by_person=avg_imb_by_person, avg_imb_by_pers_by_region=avg_imb_by_pers_by_region, name='test', all=False)
```

## Appendix - Orthogonal Distance Regression (ODR)

**Imbalance mapping** in its original form[^Nadig_2021] leverages a special type of regression called **Orthogonal Distance Regression (ODR)** (also sometimes called total least square, or Denim regression). This is implemented by default in `sihnpy`'s function. However, `sihnpy` also exposes the function for users who would like to use it.

```{warning}
I should preface this section by saying that, while I have experience with principal component analysis (which is the basis of ODR in the form it is implemented), I am by no means a mathematician so some of the math involved go way over my head and the implementation involves some (or at lot) of trust in the original implementation. However, in my tests, results I got with `sihnpy` were entirely replicated using the original R library `pracma` as well as `scipy`'s ODR implementation which makes me more confident in its implementation here.
```

### Rationale

When doing a traditional regression--which refers most of the time to an Ordinary Least Square (OLS) regression--we try to **find the slope that minimizes the error between the outcome (Y) variable and the slope**. The goal here is to obtain the best possible slope describing the relationship between predictor and outcome. However, OLS assumes that there are only measurement errors on the **outcome variable**, which can become problematic when using an OLS regression between a predictor and outcome that can both have measurement error.[^Carr_2012]

This is problematic specifically in our case. When we do structural covariance, **we correlate the values in two brain regions, which may both have measurement error**. So how do we resolve this? 

Cue in **ODR**.

Instead of minimizing the distance between the outcome variable and the slope, **ODR minimizes the distance perpendicular to the slope on both the predictor and outcome**.[^Carr_2012],[^Pallavi_2022] This has the theoretical advantage of assuming that both the predictor and outcome have error[^Carr_2012] which is an attractive option in our case. But how does one figure out the slope **minimizing the orthogonal distance between the slope and the datapoints?** I'm glad you asked. A relatively easy way of doing this is to leverage **Principal Component Analysis (PCA)**.

In a PCA analysis, the goal is generally to **create new variables that can summarize variables in a dataset but conserve as much variance as possible**. This is so we can do what is called **dimensionality reduction**; we want to reduce the number of variables we need to use in our analyses but without losing important information (i.e. variance).[^Jolliffe_2016]

The PCA will always return the same number of components as the number of variables that are input to it. 




## References

Here are the references of this section:

[^Nadig_2021]:
[^Carr_2012]: James R. Carr (2012) Orthogonal regression: a teaching perspective, International Journal of Mathematical Education in Science and Technology, 43:1, 134-143, DOI: 10.1080/0020739X.2011.573876
[^Pallavi_2022]: Pallavi, Joshi, S., Singh, D. et al. Comprehensive Review of Orthogonal Regression and Its Applications in Different Domains. Arch Computat Methods Eng 29, 4027–4047 (2022). doi: 10.1007/s11831-022-09728-5
[^Joliffe_2016]: Jolliffe IT, Cadima J. Principal component analysis: a review and recent developments. Philos Trans A Math Phys Eng Sci. 2016 Apr 13;374(2065):20150202. doi: 10.1098/rsta.2015.0202. PMID: 26953178; PMCID: PMC4792409.